In [1]:
import itertools
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import os
import random
import matplotlib.pyplot as plt

In [2]:
#数据读取
#训练集
with open ("/kaggle/input/mydataset/train_in.txt","r") as f:
    data_in = f.read()
with open ("/kaggle/input/mydataset/train_out.txt","r") as f:
    data_out = f.read()    
data_in_list = data_in.split("\n")
data_out_list = data_out.split("\n")
data_in_list = [data.split() for data in data_in_list]
data_out_list = [data.split() for data in data_out_list]

#测试集
with open ("/kaggle/input/mydataset/test_in.txt","r") as f:
    test_in = f.read()
with open ("/kaggle/input/mydataset/test_out.txt","r") as f:
    test_out = f.read()
test_in_list = test_in.split("\n")
test_out_list = test_out.split("\n")
test_in_list = [data.split() for data in test_in_list]
test_out_list = [data.split() for data in test_out_list]

In [3]:
#构造字典
words_all = list(itertools.chain.from_iterable(data_in_list))+list(itertools.chain.from_iterable(data_out_list))+list(itertools.chain.from_iterable(test_in_list))+list(itertools.chain.from_iterable(test_out_list))
words_all = set(words_all)
vocab = {j:i+1 for i ,j in enumerate(words_all)}
vocab["unk"] = 0#填充标识符
print(len(vocab)) # 9128
# print(vocab)#{'葡': 1, '籴': 2, '蓐': 3, '锰': 4, '跽': 5,

9128


In [4]:

data_in2id = [[vocab.get(word,0) for word in sen] for sen in data_in_list]
data_out2id = [[vocab.get(word,0) for word in sen] for sen in data_out_list]
#在末尾加一个32维的向量，用于填充序列
data_in2id.append([i for i in range(32)])
data_out2id.append([i for i in range(32)])
#填充序列
train_data = pad_sequence([torch.from_numpy(np.array(x)) for x in data_in2id], batch_first=True)
train_label = pad_sequence([torch.from_numpy(np.array(x)) for x in data_out2id], batch_first=True)
#删除最后一个元素
train_data = train_data[:-1]
train_label = train_label[:-1]
# print(type(train_data))#<class 'torch.Tensor'>
# print(type(train_data[1]))#<class 'torch.Tensor'>

test_in2id = [[vocab.get(word,0) for word in sen] for sen in test_in_list]
test_out2id = [[vocab.get(word,0) for word in sen] for sen in test_out_list]
test_in2id.append([i for i in range(32)])
test_out2id.append([i for i in range(32)])
#填充序列
test_data = pad_sequence([torch.from_numpy(np.array(x)) for x in test_in2id], batch_first=True)
test_label = pad_sequence([torch.from_numpy(np.array(x)) for x in test_out2id], batch_first=True)
#删除最后一个元素
test_data = test_data[:-1]
test_label = test_label[:-1]
# train_label_input = train_label.reshape(*train_label.shape, 1)

# print(train_data[1])
# tensor([5333.,  404., 9010., 7033., 6051., 1554., 4727.,    0.,    0.,    0.,
#            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
#            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
#            0.,    0.])
# print(train_data[1].shape) # [32]
# print(test_data[1].shape) # 32
# print(train_label.shape) # [770492, 32]  770492训练的数据量

In [5]:
#id转为word
dic_new = dict(zip(vocab.values(), vocab.keys()))
# print(dic_new)#{1: '葡', 2: '籴', 3: '蓐', 4: '锰', 5: '跽',

In [6]:
class MyDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
    def __getitem__(self, index):
        return self.data[index], self.label[index]
    def __len__(self):
        return len(self.data)

#训练集
train_dataset = MyDataset(train_data, train_label)
#使用DataLoader进行分批
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=False)
# for i in train_loader:
#     print(i[0].shape)
#     print(i[1].shape)
#     break

#测试集
test_dataset = MyDataset(test_data, test_label)
#使用DataLoader进行分批
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False)


In [7]:
# seq2seq模型，包括encoder和decoder，模型输入是一行对联，维度为32*32，每个词是一个元素的id
# 输出也是一行对联，维度为32*32*9132，每个词是一个元素的概率分布
# 输入仅有由对联上联正文以及填充符组成
# 输出仅有由对联下联正文以及填充符组成
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim  #隐藏层的维度，即ht，ct的维度
        self.n_layers = n_layers    #LSTM的层数
        self.embedding = nn.Embedding(input_dim, emb_dim)   #input_dim为输入的维度，即输入个数，emb_dim为词向量维度
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout) #dropout减少过拟合
        self.dropout = nn.Dropout(dropout)
    def forward(self, src):
        #src = [src len, batch size] 
        #torch.Size([33, 128]) 33为句子长度（填充后的）
        embedded = self.dropout(self.embedding(src))   # 对输入的数据进行embedding操作
        #embedded = [src len, batch size, emb dim]
        outputs, (hidden, cell) = self.rnn(embedded) 
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden(ht) = [n layers * n directions, batch size, hid dim] 
        #cell(ct) = [n layers * n directions, batch size, hid dim] 
        #outputs are always from the top hidden layer
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        input = input.unsqueeze(0)
        #input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        #embedded = [1, batch size, emb dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        prediction = self.fc_out(output.squeeze(0))
        #prediction = [batch size, output dim]
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        for t in range(0, trg_len):
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        return outputs

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#定义模型
#词的个数，embedding的维度，隐藏层的维度，隐藏层的层数
INPUT_DIM = len(vocab)
OUTPUT_DIM = len(vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 128
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
#定义模型
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

#定义损失函数
criterion = nn.CrossEntropyLoss()
#定义优化器
optimizer = Adam(model.parameters(), lr=0.001)

#打印模型
def print_model(model):
    print("模型结构")
    print(model)
    print("模型参数")
    for name, param in model.named_parameters():
        print(name, param.size())
    print("模型可训练参数量：", sum(param.numel() for param in model.parameters() if param.requires_grad))

print_model(model)


模型结构
Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(9128, 128)
    (rnn): LSTM(128, 128, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(9128, 128)
    (rnn): LSTM(128, 128, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=128, out_features=9128, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)
模型参数
encoder.embedding.weight torch.Size([9128, 128])
encoder.rnn.weight_ih_l0 torch.Size([512, 128])
encoder.rnn.weight_hh_l0 torch.Size([512, 128])
encoder.rnn.bias_ih_l0 torch.Size([512])
encoder.rnn.bias_hh_l0 torch.Size([512])
encoder.rnn.weight_ih_l1 torch.Size([512, 128])
encoder.rnn.weight_hh_l1 torch.Size([512, 128])
encoder.rnn.bias_ih_l1 torch.Size([512])
encoder.rnn.bias_hh_l1 torch.Size([512])
decoder.embedding.weight torch.Size([9128, 128])
decoder.rnn.weight_ih_l0 torch.Size([512, 128])
decoder.rnn.weight_hh_l0 torch.Size([512, 128])
decoder.rnn.bias_ih_l0 torch.Size([51

In [ ]:
#使用summary查看模型
# summary(model, [(32, 32), (32, 32)])

#加载模型
#模型权重文件是否存在
if os.path.exists("/kaggle/working/seq2seq_model.pth"):
    model.load_state_dict(torch.load("/kaggle/working/seq2seq_model.pth"))
    print("加载模型成功")

#训练
epochs = 5
loss_list = []
for epoch in range(epochs):
    model.train()
    all_loss = 0
    for i, (data, label) in enumerate(train_loader):
        #16*32->32*16
        data = data.T
        label = label.T
        data = data.long().to(device)
        label = label.long().to(device)
        # print(data.shape) # [32, 32] ,32个句子，每个句子32个词
        # print(label.shape) # [32, 32]
        output = model(data, label)
        
        # print(output.size())#([32, 16, 9128])
        optimizer.zero_grad()
        loss = criterion(output.reshape(-1, len(vocab)), label.reshape(-1))
        all_loss += loss.item()
        loss.backward()
        optimizer.step()
        if i % 100000 == 0:
            print("epoch:{}, step:{}, loss:{}".format(epoch, i, loss.item()))
            #显示训练的对联
            #找到概率最大的词
            # print(output)
            _, output_index = torch.max(output, dim=2)

            #转置
            output_index = output_index.T
            data = data.T
            label = label.T

            #第一行对联
            output_index0 = output_index[0].cpu().numpy()
            data0 = data[0].cpu().numpy()
            label0 = label[0].cpu().numpy()
            # id转词，并忽略填充的词0
            dict_res = [dic_new[i] for i in  output_index0]
            dict_res = [i for i in dict_res if i != "unk"]

            dict_in = [dic_new[i] for i in  data0]
            dict_in = [i for i in dict_in if i != "unk"]

            dict_out = [dic_new[i] for i in  label0]
            dict_out = [i for i in dict_out if i != "unk"]

            print("训练集上联：")
            print("".join(dict_in))
            print("预测下联：")
            print("".join(dict_res))
            print("训练集下联：")
            print("".join(dict_out))

            
    loss_list.append(all_loss)
    #保存模型
    torch.save(model.state_dict(), "/kaggle/working/seq2seq_model.pth")



epoch:0, step:0, loss:9.127464294433594
训练集上联：
晚风摇树树还挺
预测下联：
犀犀因獗眭燠勒勒盱犀犀犀犀洇眭兰兰尺稿燠稊辞赛燠燠刑眭刑柳崭-寄
训练集下联：
晨露润花花更红


In [ ]:
#绘制loss曲线
# print(loss_list)
plt.plot(range(len(loss_list)),loss_list)
plt.show()

In [ ]:
#测试
model.eval()
#加载模型
model.load_state_dict(torch.load("/kaggle/working/seq2seq_model.pth"))

#输入数据
# data = input("请输入上联：")
data = "玉液"
#转为id
data = [vocab.get(word,0) for word in data]
#填充序列

pake_data = [[i for i in range(32)] for j in range(15)]
pake_data.append(data)
data = pad_sequence([torch.from_numpy(np.array(x)) for x in pake_data], batch_first=True)
# print(data.size()) #torch.Size([32, 32])
data = data.T
data = data.long().to(device)
#生成假目标y
y = torch.zeros(16, 32).T.long().to(device)
# print(y.size()) #32 32

pre = model(data,y)

_, pre = torch.max(pre, dim=2)

#转置
pre = pre.T
#只截取最后一行
pre = pre[-1].cpu().numpy()
print(pre)
pre = [dic_new[i] for i in  pre]
pre = [i for i in pre if i != "unk"]
print("预测下联：")
print("".join(pre))